In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQd-QZjpwnXGDp_JJQVB9LJLhiW5Oe-l8MVhQ&usqp=CAU)twitter.com

In [ ]:
#Code by https://www.kaggle.com/mpwolke/babies-products-toys

from fastai.vision.all import *
from fastai.imports import *
from fastai.vision.data import *
from fastai import *
import numpy as np
import fastai

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
path = Path("/kaggle/input/h-and-m-personalized-fashion-recommendations/images")
path.ls()

In [ ]:
np.random.seed(42)
data = ImageDataLoaders.from_folder(path, train=".", valid_pct=0.2, item_tfms=RandomResizedCrop(512, min_scale=0.75),
                                    bs=32,batch_tfms=[*aug_transforms(size=256, max_warp=0), Normalize.from_stats(*imagenet_stats)],num_workers=0)

In [ ]:
data.show_batch(nrows=3, figsize=(7,8))

In [ ]:
data.show_batch(nrows=2, figsize=(7,8))

In [ ]:
trainSactions = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

trainSactions.head()

In [ ]:
trainSactions.dtypes

In [ ]:
# Setup notebook
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_periodogram, seasonal_plot

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

In [ ]:
#This notebook is an exercise in the Time Series course. 
#Code by Ryan Holbrook  Exercise: Seasonality

comp_dir = Path('../input/h-and-m-personalized-fashion-recommendations')

transactions = pd.read_csv(
    comp_dir / 'transactions_train.csv',
    usecols=['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'],
    dtype={
        'customer_id': 'category',
        'article_id': 'uint32',
        'price': 'float32',
        'sales_channel_id': 'uint32',
    },
    parse_dates=['t_dat'],
    infer_datetime_format=True,
)
transactions['t_dat'] = transactions.t_dat.dt.to_period('D')
transactions = transactions.set_index(['customer_id', 'price', 't_dat']).sort_index()

family_sales = (
    transactions
    .groupby(['customer_id', 't_dat'])
    .mean() 
    .unstack('customer_id')
    .loc['2020', ['price', 'article_id']]
)

#Your notebook tried to allocate more memory than is available. It has restarted.

I intended to plot a periodogram. Then I failed, again. More than one hour with only one snippet. 

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoGBxQUExYTFBMWFhYZGhkaFhYWFhYZGRgWFhYZGBYWFhgaHysiGhwoHxYWIzQjKCwxMTExGSE3PDcwOyswMS4BCwsLDw4PHRERHTAoIigxLjAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwNjAwMDAwMDAwMDAwMDAwMP/AABEIALwBDQMBIgACEQEDEQH/xAAcAAADAQEBAQEBAAAAAAAAAAAEBQYDBwIAAQj/xABJEAACAQMCAwQFCQQIBAYDAAABAgMABBESIQUGMRMiQVEHIzJhcRQkQmJygZGxwTRSodEVM1NzgpKz0wgXNbJ0k7TD0uEWQ2P/xAAaAQACAwEBAAAAAAAAAAAAAAABAgADBAUG/8QALBEAAgIBAwMDBAICAwAAAAAAAAECEQMSITEEQVETIjIFYXGBQpGhsRQjUv/aAAwDAQACEQMRAD8AU8FudSDHWieIy6dj5VO8sTkOU8qdcybBXrGtpm6rgH8vyA5FZ8dnxcRp4daC5RutTkV65xylxC3ntUcakwRdxGPFpdMLH3URy+cxLjxrHjEeu0f3DNfnIneiU+W1J/EelZjzLc76aacMjxGMeWalub5dMxHvqk5fbVAd/o/pTS4QsVuwLgD6mkbzY0fEMzN7hik/J2dTj65/OmNtn5TIPhSSfuY6WyNXOq4VfIZrXjG7xL5t+VLzKRekfVFMOMNieD76l0wadjPnhtNo2PEgVvwfuWi/Y/ShPSM+LVceLCtILjFpuPoinUtkLp3Y24KPVD3ivdmAqv8AfWPLrEwgnwWvPDptav7s00Z7kcNhbyHJl5s+DmmV3+1Dy00k5Ck9ZN5Fz+dPOISfOgB4LvTufIkYgPGrpvlEC+BJpxx0+pHuIqc45N85hUdQc035jugsAz1YgCjLJ3CoGPMCBbeJkGMOCce+qa0kHYgDxH6VONEWtmz0GKa8LkzEpz0AzRjm7glj7E7zXwTsIVmU5IfUx+JqmtbgSWy464BHxr7jcIltnX6u1KOXHKW51dFB/hTeok78oVY7X7KHhkSpCSo3bc/GpPiFjJD87z3teSPq0+4FxDtIs/GvFywlhkTr1FNHKpbCvHW4sueGm+13AI2TuD3+NacvW4uVVJScxZGn3jxrDkC7KdpCfA7U3gg7O5LDpIP40uvh/wBh0cr+gTiOq4HyQnGltz46R0rPi1vNJEtqu2NifcOlZcem7G6SQeOxp/IQ2JB1plk5Qvp9xBYXwe3e3dMyxd1fP3EVmvD5Vg7BVzJIcux8qF42TBcpMOjHvVZxsGCyDrjrUWTsSWPuT12Ejtgsa4lHdwPP31pyzYQpFiRfWZy+fM1546xhmSYeyxww/WmU9msuHz1HhVkcgjxnJLmPsbwjwJqj45FqtyfKlXP0OidXFOrY9pbEfV/SsL7M1p8omeRpvnBHup96RUwsL+RFTfJwxdkfGq70jx/N0PkRTS+QseGFW412rD6n6UL6OW7hHkTRXADqtv8AB+lBej4buPrn86StmO3uI/SWdMxqp5M3tx9n9KmPSqvrRVVyEubcfZ/SmkvahU/cxXycvrpR9c/nTWNfncn3UFyenr5vtn86ZKvzuT7qRrdjJ7C6Vfnx+yKY8bX19uPdQujN832RRnGx85gHuoNbhT2APSWPm0Y83Fe7oYtB8BX56Th6iEebitOLjFoo+zTJcC3uxzwBcW5+x+lC8LTEMje5qN4cNNux+p+lCw7Wsh+oaiW4b2FHo5izqPm5/OmxTVeSe7AoX0aRerz7z+dMOFLquZj9ama3FT2J67i1cSA/dWmPOK7wJ5nNeVjzxKT3ACtuZhm6hXyGaDGT3DLwaLbHngVuF02x+GKy5h2jhXzYUVfriFR5kUiQWz9jYrbMT+7S9FxaufMGjuPtotPiQKHlX5qfgKdAvY8cvRaLdm8lNDctsezdj0JY0ew0WbH6tCWK6LRj9U0Ircjewl5TYvcSOOmo/nVtPH31PkKkvR/DklvMn86sr5tKu3kKO7A9iG5gnMtwEH0TVSzdnACeuwFSXA07W4LHxb9atOIw50LSq92M0lSJjnNh2SfvEjAqi5bJ+Trq8BUpzAe1u0jHRcVapFpiVR44o2wNKhNzbIvydifu+Ne+XLkmBc9QKW84vqkigHxNUnB7QLGBinTdiNKjnXpHiyitWnKsmqAfZrfnqPNvmlnJcvqR99BcB7i3lqPF83uJ/Oqz0hfsoqe5ej+eSH31RekP9mFST9yBHhjLg3DuztLdw2e1ikYjHslGQDBz0wx8PKteU+W1iWBjK3aXWt4wEBjRQusdp3gzZGPZx1x4ZrXh0ymxsQGBIglyARkd+LqPCjOHxpPb2JDr2UUbpO+Y/VFVT2hJkDvR43HQ+/NXaVfBQ5uuRHx7lSC5DT3EsiKJ0tlWEIT2rSLGWZpB7AdvLOFJ8cBrwfgTWy3UIdW+TxhlYqe+GV2GQGGNlA+Oa0jtjLbtBFpaRL2GUp3UYQLPGwlKHGO4hJwOobbO1MpblJJL4RsrGWMRxYIxI8cTBlRs4Jy2MZ+i3kaKiqQHKVsVcG5WWGeMLIT2yCRiy+ySrEhQCNthQfB4Q811M7YEMUcjaVyWyrlgoLjHsbAnx61SR3KLdwKzqCkSK+WXAfRJlCc4DDGce8VP8GQq/E4Tp7RraMIvaRd8ss6qFbVpySPEjqM4qaVfBNbrkz4hwow3KzFw6yR9ouEKEAdVILNk4K758fxI43axRyq0srCZIe1SNUGiVcnMUY1azJsBnplht4UbxyYNLbojKzxw4YBgwDHSACQfNDWzBjG6yGN7cQh4ZQAGiuclRBGGZmLdO6R46dwdIVRTk9hnN6VuKuZuExzWiTXLyRtGyKUjRT61gpC+2wKEsNwelbLwATw28byNG80faR6UVlTCBwsveyxIP0cDbHkTpza6mzkTK6vlMJ0gjOAsR6daZcOx8yk1KI4IWWdyygRssaph8nbJz9wz5UyivArlLyCWdrm2GtgutkjwELlWdFOWOtcgEnoKHt+GlsWrMBrkeJnA8ESU6lGdieyBwc41Y360VLxIR2mVK69asFcKWUiBXB0sDhgQN8bb1lwObv2ryPuZnZmYgZZ4LhiT4DJY7DYZwMDAoVG0hk5U2YcJsktoswuXTtXhHaJhhKhfUzFHwy4Q4xpPTz2+gtZI7jiCxsmqHsShdGIZp01HUA42B8jRt3lkBdgZBNNpClGJgBkcZWPIwdKEHGr2QTk4PpoCJr6XK9ncm0FuwdWEmhAH0hSTgdSfIE9Bmm0oTVLyKoeFkXUsysO9bxSLqVsdrO5jiiOG/eVgd8javUlgZrqwcuAZ4i5whAXEXaYALnx99MuG3QKQplSxuZFYZGrQnbSpkeQZsj7qGsbhO34Sda4W3IJ1DY/JjsfKhpj4Dqku4LxSGSTsc6dQu1tsBWAz2Gp33Y7doGx9XHU7k/iSKC8ZlXVE8QKsoXWJSFBi75OASdiCe713zQov1Nrw2V2BlNxA85yMsy2rqZWHgGRFb4Gj+IRoJblndSXeCSDMgLaT2Ub9mA39X3V8Nyz9c1NEd9ia5bbmHMkcbQO8rOsUXZ6uzAMjPK4RQNRCgDUuc5zq8MbkScJYa7YuMLG0gbSclQVCAjVsc6weucAjGcDLj1k81vPBHhpXNvIiZALLFNGz41YBwEOR7x5ituGW8cMrxJKGXsZF1sYxmTMYZdShQwGV+BLD4RRXgLlLfcCnCPbPrLCNHjicRqDIZJREwILMFCBZkJ6k74xjfWPhRaR7JyQg0osqqO85RpGGC+VxH2f0SM6t/AYQRNLBcRInrDcwOE1R5MccdqjupDFWUGN84OdvhTdLlDd9pqXQZzpfUNJxZ6Dg9PaVl+KmoopdgOT8iLlWwVHuBE+YoUBDyr3mkVpRKDpcYUaEwcfSPlTO5R5haxlkAuQuoqjZUNbSynTlz4xqN/DPmMC8n3IX5adaqSX0EldzrlwVB2PhR54iry8PkaRCTpaQ5UYZrS5znGy7k7VFFNcElKSfJP8AL3DYxPKsLuY4o2Ys0Y1s8bFXVAGx+7gn3jHiHFvdLJh1fWpjRw2nSRrB7rblSwx1XbcbbZIVks/yqUtLHFKVcwsvZEP3z2cY0MFJYBiQ+SdIP0chnxLAyQFDmMtMq4AWQ4+iCdJbLk7/AER8SulKPAyk3LkjeBL2t47fWq6mXdR5VG8hpmR2+saspT3m9wquKL5MjJPW3zfV2qn4hcdnpA8qmOWxquZD9Y1QcwDvr8Ki4sEvBI83N82zU/ya+IyPeaf8zpm1qc5V2Q/E1uydOrqCMsMr5kMeWUHymQnzpv6Rm9QnxFAcvrmU465r36SJiojXr02oPpYrd8hWZ8IN5aGIWIH0T+Va8ioDliBnUSD4jfwPhWXBX+bsW2Gk/lRXIadw4OdzvU6lVFEw7yZHek4hrpQRnf8AOug8JXTZtnp2fT7q53zt379F+sK6Ncdyyk+x+lZOyRf/ACYm5GjHZE4GMsceHjRPLMQLyNge0ay5R7tvq+qTWHLvHIIw+uQA5balUW+PIzaQXy+M3Ep+vREyg8RJxuEAz4+WM0PyZIHkdxuC5INEW5zfynyAFI+WRcA3Pag3NqMb5zXjmjBltxtkEYPiPhWvN+99bjyUmsuKOpuYg3hvWjHBzlpRXOWlWPeOKBbE4GdhnxpNzF+wMPcKq75UNsSQCMVK8xDNsE8yBTSwuM0hYZVKDY55DjC2wwMd39Kx5UUapDjcs2T4nfxPjTPlGBRCF8hj+FYcGtwjuB01GplwyhyDHkjK6FPK6D5TOfHXWVyo/pFhgdBRHKv7RP8AbNDT/wDUn+ArO+C9cm/MP7ZB56aeXsQAjwAN87Dx8/jUrzrdmO5hYeVHDjBIQudhTKLaZqx9HPJDUuDL0lbrADuNQ60y4Sga3wRnAGKnfSFxRHWLT4Gm/LPEUMJGfCpKLuitdPNLeLGXHYwbMjAwMHGNqC4umqxbx7v5CmPE97N/hQSDXZMPqfpRXP6KJKl+xZ6PCCoBA2o70iIoiAAAywzsKT+jyXcj3/rTn0kj1Kn3ilXxC/kgHkcDWRgYI3HgfiPGqXiagJIAABpOANgPgKleTHxKKsOJp3X96n8qGP4hyL3Ep6PBu3xP51WSjvP8P0qS9H7Ydh9Y/nVhKO+w91PDsLPlkXyifnD/AGj+dUnHl74+FS/Ljabpx9Y/nVhxeLJU+6lXxf5A+SD5iPzU1K8sv3WHvqn5i/ZDUly02z12H8kYOxVcqx+sz76w9JEnrUHwo/lNO9SbnyTVcKPeKk+CRHELhbSQ/V/SpjhPOEsSaIwBjO9PuLyabJvhXP7VtjSZIJtWGEmroO4fePcXqM5ydVdZ5mbRYP7wBXJ+RYtd4vurpvpIuuzslHmRWCa91I0wfttgUMnZ2LHp3P0rmUTe03xqjHMbzw/J1UDI3J8AOpoJOXiEI7Vd/qn+daMEHHkTJLU9ig5E5oggQK+c0/5WvVnuZpE3BIxXOjwhoVDFgwzjYEYPUbfdV/6JYQUZsb6qj6VSVrnknruPITzHvxGMeSULfb3iDGcCm/MMKfLVbG+mlCvm+2PQUcGNwyJP7AyzUoWWl1bg2+npnFIOYrMpbo43wRt50/4s5EAx5ik/N0pFsmOuoVujjUpKT8mJZJRWlDflqZCm40nH6Uv4ZfRK7jV9I0x5aIMQ1DBx1+6ohCPlEuP3jWXq0nKjq/Semjnk03whxyk2Z5iP36Eu5AOJPk42FKuXuLvHNIBjdqF41e6rvUTucZrnOOx1Y/TpOfNIO9IkgM8JztjrXjiMmIFK79KB5rky0Q60bdKPk428qeqtI6/TYljio2KOZ3zFG3vplyhPlcUp5kyYU9xo3kqTwo/yQW/fJHRrBw0DKfI5rLg8OqBk9xFYcFUhXB6HNfctzHvLjxNVX7kcnNgi9X5sneVI+yuXjz0Y/nT/ANJH7MG94pH2Rjvmz4nNUnN9t2tk/uGfwoJbNGLNjakmlsS3KcvrV+6ugXq5B96n8q5py6+GQ/Cuq2aK4XPlSY3syvOtLs5/yU2Lh1+sfzq2m/rPiKlkhWG+dVGBn86rZYSzIR5VZCW1/cqm+/2OfW7aL5x9auiJbh1BPhULzPw8w3mrPtYNW/C5cxg02Om2hZvZM5nzD+yGo3l1vbqz4tvaN8KieX27zCurP5IxR4Oh8pJ41K8zvqux9qq/lX2T8Ki+JHN4PtGpPsSI35pbFn8ag4h3Cat+dXxaqPOocn1ZoZOf0SIbyVxRLecyOCfKqrmzmRL4RwICoGSx8cDy9/SueWh3p3wVvXgfUb81rOsca1dy3U+Ai4swkixwFg5B1MSDpT4ADfp+I88j9MVvob1cs2PakGo7/EED8a0kU9tcKPbaMaP8gG33kUbwy5QW6jUo0gBgSBg+JIPmcn76s8/gQBt4lEYZXZ7ckZGe8h6bbdN+nw8810r0d2CxJlCcFvHB+8EAVzm0ZexuXHsM3d952yR9+Pw91dR5DjItYs9e7n46asx7ITJweLlRJePq3CAbfHPX8KU8P4jG16V7EYx1wufwx+tN7RAbi4I65UfwapzgePlr9dWMEeIPlinWCWvU+BXkWnSXl/CzwgDAGrxJ291JOeIHW2Xpsy7jw38ac8WhTsFeQtpyNh8cAmlPN5X5EdGrT4aviDt7qeEmq/JU0OeW7eQRL7JBXPjtt8Kiru3ZLiRiBgsemdj5Grfhg9TGM7Ffx7o61HzNkHP7/wCRasnUNuR3Por0zdeAC1RIXJ7MMSd2OOvXc4OB5CgON3UbzhBEFclQG2wAT1yMH7vfW3DL+WOZgw7v0dQI28AG/Q1rxWCOZzJoCMASCPHSM9fpCsb4O8tqb88mvMnDMxwKukMCCWI6jB2zjPUj8K9x2rSQYGABsSemfIUFzsMW1sfAMn3dxqZW5J4e2M9D06417/wzRXLDDJJXv3EXNPD3WDOzAYzjOR7yD4V+8kWUhw2wB6Zzkjzx5VuxzZS+Wl8eWNG+PvzWVu+Fh/d7ufL2ds0JfNCuctbVnQuBIyvpbGCCRj4UBwZ/nUyjc622onlhvWjOfZ2z5Y2xnwrLh86C9mRRpfJySM59436ZxttQyKpL8mKbanJfYQ8ecx3xyPAHrVfBJ2ts+fFD+VSvO8RWZGPU5B+Oap+VnDQ492KqjtOSGy+7BGRB8KQ5GB0b9a6twWJuzU1z+5URysoGMNV3ytd648eVZMOR6pRMfWR9kZIl+OWrfLiR0IBqvFuezVs7ip7mhSLpTnqtUnDH1Q7+VWQk7cTDN+1EX6RVPaxP4EY++nHLVxmEe6vfN1sjJEXGwNF8KnRUwAMVpxQeqyt5PbRzFZg8DL7qieEPplZabcP4joPZsfcDSG+VoZ9RGxPX3GunKWyZnijqXLD90/D9Kjbr9s+81R8ocQXs2bw05qZuJg1zrHnTSfBEhhz63zeMVGSn1dVvPr5hjqQY5jNCb3f4DBbGNj1o63veyl16dXdIxnHXG+cHyoCzNft03fqpP2jdxpxHjGt0dU0Ov0tWcj90jSMjc/iaLueIwMgaSA6vHSdifM7ip9juKIuW7gpttwDiLiYfGUAjTpGDgHHTJx/DFdc5FujJbqdOnx65/SuJWB7prtHo82tl+FXwScSrIwCCVluJ2U+P3H41hy/xd2uXGldvHf8AKvyOT1s5+saX8mNm6kPvrdOK0ozRbtnQuJTNHHGBgjPjnakHPd23ycKQDqIHjtuOlPOYD3YvtVOc+v6pPtCs+KCpfsaTdlPy4xeJcgbLgY+H/wBVEtckzSRYGFc7756n+dW/Kp9SPh+lQKH5zMfrGsXU7TO/9CWqcr8A3C+Lydu6nS2/U5z9+DWHMXEZGuFQ4CkDOOp+JPhWHCzm4k+1WfHW+dr8KxN7HofTiknXcZcw8TaKKIaQwztuQR/OjeG3ziF2GO9liu+MnqR40n5w/qYviKYcJOYD8KZ/J/geOOOt7eAHjnEXkgI2A8cZyfcT5UBy3xl0ZUwCBsM5yB5ZHhW16PUNSXhLd9fjSTe8WCcIqdUdc4LdO0iscdMAY8/40rvZDHxEvkZO+PDfrRfL77pS/mpwt6h8SKGfizJLHFZq8oO59iLRpJtnIziieRp8riv3mdNVnnywaW8jz94ikbrIn5K4q+ncfBtzVDpnJ896o+R5sqRSnnWP2HrXkObvYrBWnqGjH1C1dNfg19IC6Xhf34p9y8+YqTekpfUo3kwo3k2bMeKujtk/KOY/hZ65nh1W5+qc/hSqwmBQEVQ8Ui1RSL7jURwi47hHkSK1QdblRzDmWz7OQ+XhWUUwmj0N1FUXM9t2kQkHXFRAlKNmulL2srW6LHkJsM0R6bj8aU8YHye6KHoTt99GcqXIE6sPEj866xd8QW04TbXC28UjdnaqVdR3hIEVt8e1v796WctKVEjuct5zhJtY5PAdajreUYIJFf0FzjwmJLm07ONVVpomZVAC5S5gQEKBgZEpz56R5VvxO0j/AKatF7NMG2uMjSuD306jFVue9jUfzbqwa9yvnfNdxn5gjtuOy2nyZJBctaqGJCiLEWCQmg6s6vMdK+9NnMcdrF8jW1RjcRP60EKY8MBsoQ6vxFLq7BOGs3Q1tNICg3FdJ/4dole5uQyqw7JdmAP/AOz31rxjjltDHxy2kCiV5m7FQm514Q6TjAC4yd+lNq5JRzaymGCMiuv+jXiatb6c7qP0q1XhMMvD1hMaBpbbQNlDEtDg6T1zvnNCejbh8I4dao0cZdoVlIKqTiUl9RyPNjVkM+lcCThqIKa4w0p95ofkBsyu+c5asbY9neW0Z9l5LXA8CGeLbH310nmvhSLc2kqxqFeRInUIME6xJGWwMAALLufFh9+/P1Chp25VlEMd2YczS4EPxqW50vFfsUVlYlh0IP5VdNwyJ7wRtEhjUyyacAL2iR2qplehGJXOD4qp8K922m7NxBLGmlG9WQuGUiSaMMD+8DDqBGPa91ZY9XpSVDvDbuzLgS6IRnrj9K5zby+umP1mronLCo2mN4omLCWXXpGrMkok0lSuwHbY6n2ajueeOq8s0KwJELcyLrUjMmQvVQg09PM1lzTcnb2Ox9Hk8eSoq7W/2XkmeXyDK7fWNCcSm1XZ92BVh6IYkW1vrqQKQijd1DBTFG8jNuPJ1z9kV69LFqi3FnKiKvaRsCFAHsMjdB/e1nr22dSPXa86w135smecXAjiHvplwVx2BwQcDfB6bVT+jvh0U1zLJIiuYY4uyyAQrSGTUwB+kBGoB8NTedMTIL6xunmjjV4nmWJ0XBXQgdD1JzuARnDAdN8UyW9gzfUVizOOm0mk3ZzOY5t3++kPDCNS711j5Mh5eZ9C6uzfvaRq/rz44zW3pHtUWytCqICZockKATmN852pXG0hJfU3LIvb3a5AuXXGUpVzncA3iAHoKtebYEjntWSGM4MpKEBVfuomGIU/v56HpRHEeIxxWS3YtYSSI/V7BR2jKp7+gnbP7u/uo5IqS5KH185TU1HnZb9xPetqsm+zU9ybJiQfCrDlFkJSF7eHvi4lDgAkevRtBBQbfOMZz9DpvsBPepLxGKH5PGixSSLkYPaZA9pdAA9jzPWllBNxdirrJx1Q0/d78GnOW8CnyNBckSYkpjz5cRxhoEhhBYRnVsrjMhOyhNx6vrnx91GXDxxW1mewhbtOzRu0RenyeSXA+sTGFHXduhqieBTy6r4ool1L9DS4833MfSY4+TD3sMV+cjybY9wpX6SIFieKKMBUfDdmowqspILKo6asrnH7uepOWPJi4NUz9uZIz1/1WUl50b4H8q5vw98GQfXNdB4tLhHx1wagbGMqDnxJNaIvYpoluXLkT2+k9QKkeNWmlyKO5Ev9LlCdjTLm2yz3xXW+UbKeHQh4DMVkT4iu6zcae24LaTxqjMIrTaQErhlTPQjBx0PhXBuHe2B5Guv8C5wxawW72ccqxRxJl5ThjCqhX0mIgHK5xk486qnFtKgp0Pua7FI7iCRdWZJYC+p2bdLy2VdOonSPWt3RgZOcZJzvxT/rdp/4W4/746l+Zubu1Ku4CaN4wozpcOjhyze0Q0cZHdAGnfOdgJ/TNGrCV7BZJ0Uokqy6VKMVL9VJQEqDp73TrVbTGMeaN+Z4fdLb/wCktY/8SB+cWv8Adv8A94qQt+a5H4h/SMqB5BIspjB0qQmAqBsHACqBnB6Vp6Sud/6TkifsOx7JWXHadpq1HOc6Fx0qSXDIVP8Aw3/tVz/dL/qVJekP/qN//fSV69G3O/8ARkksvYdt2iBMdp2enDas50Nml3HOKfKri4uNHZ9q7Po1atOrw1YGfwFGPJGf0G932Y4R5Oyof8VlKQP8wWteDYTiEsC+xFZ2qqNgAe0nyQB0yAn4VyPiPpUaZbFFtdBtJYpM9tntBEhQp/VjQGBO++M9DRfD/So0V9cXRtA3yhYUEfbkCPsVK+32R1ZznoMe+goN8EsL4va/OOGyD+1tAf8APFXTnu1kvXtH+jFBcR7j2kmcPgdcArDn7dc149cdnBazadRhaJ9OcauxZG06sHGdOM48elFcF5tluuIJeiARYj7ExmUsHXLkkvoGnd1Psn2B51u6jDOeml2KMc0k78nRIGHy1h4lZT9wSyz+YoTlT+vuvtH/ANVdmg+LX7xMbkKgkBJC5LAoyRq8ZYgEBuyU5A2KqdwCDla8aklhkkhijgZ1LSOJGdg2N9AKKPFsMTgE50nJrL6GSrrYs1x8g3IV9ru+zz7Fv+bRD9Khed203N8fOVvyFL+VeeGsb2aTsO21L2YXtNGkBgc50NnpjFZ83cVMxeUpoMrFygbVpyemrAz+AqvqXu6Ox9KjKMpTran/AGdD9FvDGbgsqoBqnabGo9090Qn4DEZH41++kvh7iwsTJgyRvGkpG4yYWDkE7ka1X8an+VubmHDxZLAqhVIMnanJ1SF3OjR1OSPa8az/APzZri1SxMAHZ9mO2MpYnsmBLaNHVgCPa21eNU6lVBxdLnWaOTTy7/RU+iWUGe8HiFt//e/kaL5VHzDiH95cf6C1zblrmyazu5JIgro4CyRtnDBSSpBG4YZODv7R28rTmHmKFrLs4Z47NpVd54jFI7M8iguiSYA66l1Y3GMacUYyTQOswZPWbraT5/BtE2eW2I/s5P4TsD+VE+kz9is/7+H/AEnqV4Bzf8nsTBNAs8BzlC2lgJCWcDYhwWJODjGTv0A/eLc2S8RMXqRDDG2pE1aizgEKzNpGMDbAG2T12xLVBXQ5vW06e7d/Yvucx6229yyn+MVBcd34NF8IP9RKD4nx1rjEjRqgjRxtIXyXKHxRcY0fxpJbc2tcWqWXYBVTs17XtSSeyYHOjQMZ0/vbZ8aSc4q032Hx9JmqHt4dv7IquUkxPAP/AOFx/qWlYQQL8u1/S+UMP4vX6bxrd4pVQPpjkQqWK/1jRNkEK3TsumPGlQ4o63Al0DUXLlNZwMlttWnP0uuml1xSS8Cvp8k5zkls06G/pC4h3vk+lcHsZNWMv3JteAc7A9kB/iNMpr54ray0BDr0Btalu6lrJMdOCMN6oAHwz0NTfFub/lGqH5LFlgF7XtdTKAc7AxDPjtkU77GWSG3Qqidl0OtmLfNpYRtoGkZkB8enjSRzQ1ve+P0Y8uKcIRUo1z35FvNnAALlHDSFShOJHaQghuis5J097OnOAc460by82JMUPx3ixkm0lVUoCO65bOTnxVcdPfRPAl9YKxZMifUe0Ol+nuMuKHOV86TnhwHSm8+7GvlirqY8aoyOR/MXCpykqsK6IQJYt/KoPhaLvn2vCrDhUpCgVuxbIrkLrXhGmQnwqp4GCDg0FcRnqK+tONiL21z8KZOK2YGmxpzHaAoa5lxaEAmrnivNsTrgK2aib2TWxypGelVzcRlYFZHes7xMGt3i0Niv2+TxpauIe4CtHQLhDQYoxz3QBQgRmEI3ou5OCp94/OvFtDX12NwBViTSAzpE83bW8a+AAzTvkvh3eDY2pJytbGREHhgV0Dh1oIl2FdWWSsdGKt6FnN83d0DqdqJ5cjxEU+rigL9DJNnwX86acBG7VJpLDQE7mcYv7LTeS5/eNC8Um1sFHhVBzyBFdSeZpDw231vmuD1HzaPY9JG8MYx78lFYDs7dm8xS2xlEcbP9Js4o7jp0xLGOppe9mwAyNvCsre52GmmkuyPHCoi0g8yc0dzI+ZQP3Rii+D8N0gyNtjpS26bW+fM0GqVDxjSPNxKWRYx4mq/hFiI41GOgpNy/w9deqT/CDVIzhSMnY1anSsTdW3yfcak7ODSPaehuUuHd9R95r84jKZHz9EbLT/li10gufAVkfvyfZFeSfp4W3yzHmXiiq+gbsB08qmuK35jXSDmR/wCArVJ+0uJHYeJxRfB+BmSUyPuSdvcKoyZN7M7lHHiX+QvknghOGfr1NXEcg3PgorGC27KPAG5rHiMnZW7nxII/GljFx3Zw8+X1Z/6JO1ftJ5H+sarOBR7lvIVO8GtNCA+J3/GqqBNEQ82qrpoOeXUDqJpRpHpBk1sFrOFdqY28AxvXo4pJHMkz+SAxG4prw7mB49m3FbtwlfKsn4MPA1E2uBnQ3HOUeMaDS++5mVvZT8aXT8JcdN6DkgYdQajk2FUFxcVYNnAPuo6Xi8bqcrhvCkVaQMAwLDIzuKWwnrWScmjJFytN+J8PhkiE0S6CB7Pn76W28ZYYHWrINUKxU64rRJNq3u7RlOCpofQQaXhhN4ZiKO4YFdjkd7wpaKL4bcFJVIGdxtVuOdSViyVrY67ybaYjGPCn15xkKNBGD0pbyjcIe6SA2Blc1+8y6VKnUBv5iuj7ZSMm5jaO+ohuhOxqm4Xb6RmpxdwCKJs+MOoYbMB191HK3KNIWKp2QPpJUNdnHlvQnL9t3h7qP5psJnnM4X1Z8fKieGWgRNXmK43VLTJs9p9IcZ41XYUXx7SYeQ2ppxKILH8KC4fFmY52waL49L3DWHsdtbWxPdcekA0DGkih4ISAGBrxJb5xtk0XaQEHOdvKhJmNykm2HwX2vCMulvA+dVFpwppIwzHpUbJdhj0wR0xTew43Kg2b7jQb7lWTqJVuP7W31OqEYqmu1EcJA8RioG3nllkEgJBHlVVLdvIqg0jlUGu5hz9UpyS7IXWlgCelVfC7URrqO1C8JsfE0RxNyw0r0FZoY9PG7MXUdS8j+w4QApqPTr91TfEbnt20qO4p/E15iac91mOjp91NLO1QYC1e8c3GmqRjUoxdrdgthZaiB5UzuTuF8q3CBB76GRcmr+lwqL2EyTb5CbSLJpiRQyjQmfE1mtwRXRUW+DI2TI9Elp/bXP8Amh/2q+/5S2n9vc/5of8Aaq7r6q7LqIT/AJS2f9tc/wCaH/ary/ogsj1luD/ih/2qvK+oWSjm8voN4exz210Pg8H+1Xj/AJD8P/t7v/zIP9mumV9UCQdr6I7RE0Ce5I97w/pFQ9v6F7FH1ia6J8i8OP4RV0SvqhCAuvQ/Zucme6HwaD9YjQsnoPsW3M93/ng/2a6TX1S2Q5RF6JOGNIYxNejBK6swhC6jLLq7L2sAnHkPhX7ZeijhoKSGa8TJUx9o0ADhiNBGIuhJAwcEEgEDIz0eLhsav2oXvE+ZxlvaIGcZPnWL8vQHTlWOkgrl3OnQQyAZOwUgYHx8zmWyERxPk7h8kolW5uwShPqHh06VG5AaPU+2D3c4BUnAIJ/bz0fcPkALX113cDHaw6id8ZBi65BHxBHUVcvwSE9Q2O7ka3wdAAXIzg40ivK8BhByA4PmJHBxqLFcg+yWZiR0JNNql5BpRK29jZW8ax9rO6hA2stCTjv7bAEn1b50ggY3NEy8CslDfOJO+yhtLISC0ix79zbDMM56eNUkHCYVUoqYVl0kAnGnvHA32GXY7edeJOAwNqypwzFiNb4LM4kJxnGcgfdt0pvVn5F0x8Ce84BZyW6xds6ox0hlZMlgcfSU+PupPDyfYRj9um0kfSeLGME5/q9tlY/AE9N6sf6Ij29vbO+t8ncEgnOSDsCD1CqPCvw8BhIKlWIYYbLuSR2bR7knPsuw/DyFVv3cmjHmy4lWN0St16MrVQXa4uFHidUXUnAA9X1JIAHvoUej6wbA+Wztk4HrISCcgbHs99zjPnV9PZo4IIxkgkrs2UIKnUN9sCgH4DD2mcPup1AyOQ+Cuz5PeG2+evjnAwmmPgtfW9Q/5sg4eSLFkWZZb3QdQ1BrfAKoXII0ZOw8M4OxwcgbHk6wAcma7GhX1D1JOqMgPGoVCWcFgNsgk7E1eJweJcqFOCpU5ZidL6VIyTnpEgHlivDcBgPtR6srpOok+02pjufaLb565J8zUcYvsD/lZv8A0yCHJHDkJHbXRxII/btidRDH2Quoew2xGT9EGmnD/R7ZyAlZbkAEe0YhkFQysPV9CCD5/CquXgcJ6qfLGt8BSCCoGdlwx26fgMF2dkkQIQEAnO5J8AoAz0AAAAG1D04eBH1GSXMmIrDka3iGFaQ/Ep+iijU5aiHi/wCK/wDxpxX1T04eCtyk+WAjhSYxlv4fyr7+i0xjLfw/lR1fUYwjB2kBu+RceDIfpP8Aiv8AKvVvwhEOQzfeR/Kj6+p5e7ZipVwCvYqTklv4fyr5LBRvk/w/lRVfUIrTwF78g81qG6kj4Y/lWf8ARq+bfiP5UZX1OpSXcXSj/9k=',width=400,height=400)

In [ ]:
customers = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")

customers.head()

In [ ]:
#Code by Lucas Abrahão https://www.kaggle.com/lucasabrahao/trabalho-manufatura-an-lise-de-dados-no-brasil

customers["club_member_status"].value_counts().plot.barh(color=['chartreuse', '#f5005a', 'blue'], title='Club Member Status');

In [ ]:
#Code by Lucas Abrahão https://www.kaggle.com/lucasabrahao/trabalho-manufatura-an-lise-de-dados-no-brasil

customers["fashion_news_frequency"].value_counts().plot.barh(color=['magenta', 'purple', 'darkgreen', 'orange'], title='Fashion News Frequency');

In [ ]:
articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")

articles.head(2)

In [ ]:
#Codes by Pooja Jain https://www.kaggle.com/jainpooja/av-guided-hackathon-predict-youtube-likes/notebook

import matplotlib.pyplot as plt
import seaborn as sns

text_cols = ['product_type_name', 'product_group_name', 'index_name', 'section_name']

from wordcloud import WordCloud, STOPWORDS

wc = WordCloud(stopwords = set(list(STOPWORDS) + ['|']), random_state = 42, background_color='pink',colormap="Dark2",)
fig, axes = plt.subplots(2,2, figsize=(20, 12))
axes = [ax for axes_row in axes for ax in axes_row]

for i, c in enumerate(text_cols):
  op = wc.generate(str(articles[c]))
  _ = axes[i].imshow(op)
  _ = axes[i].set_title(c.upper(), fontsize=24)
  _ = axes[i].axis('off')

#_ = fig.delaxes(axes[3])
_ = axes[i].axis('off')

#Much Better than asking for Votes: Shopping!

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoGBxQTExYTFBQWFxYYGB8YGBkYGRkZGRkZGRgZGRoYGRkZHyoiGSEnHxYZIzQkJysuMTExGCE2OzYwOiowMS4BCwsLDw4PHRERHTMnIigzMDIwMDAwMDAwMjAuMDAwMDAwMDAwMjAwMDAwMDAwMDAwMDAwMDAwMDA4MDAwMDAwMP/AABEIALkBEQMBIgACEQEDEQH/xAAcAAACAwEBAQEAAAAAAAAAAAAEBQMGBwIAAQj/xABKEAACAQIEAgcEBQgIBQQDAAABAhEAAwQSITEFQQYTIlFhcYEykaGxByNCUsEUM2JyktHh8BUWNHOCorLSU1STwvEXQ4PDCJTT/8QAGgEAAgMBAQAAAAAAAAAAAAAAAgMAAQQFBv/EAC4RAAICAQMCBAUEAwEAAAAAAAABAhEDEiExBEETIjJhUXGBwfAUkaGxBdHxM//aAAwDAQACEQMRAD8AVWLdFolRWR4H3UTbPgalg0S21ohFqK2wqdGFUWSotTItRIwohKhDtFqZUrlBUyChbCQNdSvmDX84Y5D8anuio7EjNHeKXkflDgtzjh75i08oAo9YFxQPGaFwZAz+Ypjh4NwGPsmayPkehuDHVjw391NL57B93vpXhWHPaAJ9802IBUDcTVYns/kDk5R0dFEDuqRdq5burutePeT9lQhnzLpFeYaV1XymNL7FHydK+sK8tfTUq4kIbA7Md0ihbC9lh+sPf/5ou0sA0NlgMedZMnpXyGR5ZVsYgyODpDGlyjf+eVNMWBFzNpLT76WoKnTvf6B5VsRsKjIqdhXBFbEzMQOKVcZX2fX8KcOKWcaXRfWoyAGHGtOcMulKMMNad4UaUqXIaCLa0QiVzaWiUFRFnVlKMw6VDaWjMKlEigjLX2iOor1EUUcYDaJA86Lt4SORqvcA4317qiMkbtmkHLzIk71auGp1jKVNprefKTmhojdddTMelBoQetnxMP4H4V1+TjuPuFWK3wyzqZGUb9/vBqu4zpFbt4qxYGHUpeYhbrXGUKFWZYQd4IA8qKitRzdwi+vLSg8QyoFLFVEkSSFE90mln0udKhgitmwV664AwI1Ftds2u5JmOWhPnldviV66Mty47qWzdo5oY7mTr6ULTW4aqSo2mwQ2qkMP0SD8qmUVkPC7+Ita2WdfFf51q1cJ6c3UOTFJmH3x2XHiQdGqvEfdF+EuzLhcrnDH2h+kPlUeGxtu8ue06uveDse4jcHwNSYdQQf1vwqpu42gYpqVMkwWmaebUfaQZj3BaAwCyhH6UA+tGWDDPH3Y+dZXyOG+FykKeWnypxaaQCNqQcOfsgRpTmy+VZPuqQdIHIrJ5kiDUtB4dxM92lFo01qwSTXuIkqOq8a+1wzRT5NJbgnudfTUF1zoRtUuekrJHdBUfEET50M4gNrMzXaXO0fOKExVyAxHI/Os2Sa0pfMZCLsQX2AdzvPKl1vWjgRmbmZ1900DhwZM95+dTp+UMy8M8wrlhR2GwLXNdl2zHae4d58BSXjXFVtkrZRrxGhbRbc9wYnteY08a0yyxjyxUMcp8ImYUt4yOyvn+FQYDpIHfJdtm2TsZDJ6kbUXxpeyvn+FHGcZcMGeOUPUhfht6d4QaUmww1p3hBpQS5Igu0KKtioLQoq2KiIEWkphgbUkUJZWm3DbetGimMOqFeqSvUdAn5/wHQxXtLdXH4UEx2S0MJGx13qPh3Ar7sVS/ZlZ162Nu6q5ZWj7C0OxCwWGxS/bePBgfxo1brC2btx9E7RDb6DekFha+9KcUbeDYA/nGCfAsfgPjVFpWymcc4m+IvPeckljpPJRooHgABVu+jXgtvEEs+uXl/O+1UKtM+ivDMqG7MKdB4wTVZdoj8O8jUeEcMs20hbajv0+dQcW4LYuqVa0hB8Bp5HlS/EdKbdohFBdyJKA6gRMsFByiNdYovF4+4bHWoksfZXfU7c6RtRpoy+zn4fjimY5MwDeNttifL5rWhovZ/xH5CqB06uXTds3LyZXZGBOkEAqREKDoXIgz51cuD3w+HtzvGp8RA/CpJ7UKcd7GvDwBb02zVPgCPrJ3ofCLltKeU13gkGR2ny+VKZEO8GRkBG9MsU4CAHn+NLrFuAsbaUbePaHdB+FDxsU1dBeEAy7VPUOHELHf+NJulPF+rHVqdTv3+QrZCShCxGlylSCuIceS3IXtEe6kL9L7wljaBQa5ROaB+JMCKDw2IUEG4YYnQHWWO2o09DroKcW8IIMry93cKRKcpdzTHFFdgng3SVL5ClShO0mQdSInv0p2oBEVTLdhVEruGMerGPjVt4feDoG5ka+dXjbbpickFHg8zgULiIGYc96mxh0gd+tDXXzNoNxr7qRk5oKC7iV4DsOc/GKFwFkSxcwqyWM7CaIvjtsP0hJqpdMMa4R8PbPtN2iNyTy9FI9Se6hhLSk0O0a3Q0x/GWxZFmwciEat9y2eQH3mj3T3ateH8AsKNV6wjmxzEeQ2HoBSzotwcJZiYuMJJG4JEaeQ0HlRXDODtYulzcd52zGYFNjHu92O42R1xboxhbqmbKg8mUZWHiCKr+NtFLXVkzkbLPMiJUn0+VNcfx64l1A6nq3OUECRJMAnTx76Q3McL1zExslxF9ytPxpkXUlQrMrgznD708wg0pLh96d4XanMxIOtUXZWhbIoyyKiIGWBTnh6c6U4cU6wI7NMiCwmvV6vUwo/L1haMsihbAo6xSyBdhdRQPTYHqbI5Z295A/jTLB2yzAAEnuAk+6jekHRDGYiyos4e4zBwQCAmhBB1cgVVBRdMoXRrhi37xtsQDByg6Anu84n3GtL6PYYYQGxqVmVJ7jyPcZpHhPoe4mwJy2rTZgwLXRIjXe2G1q84PoVxB8z4h8MXZp7D3SNFUfbQkSVJOu5J8KDLCUt0aMWSEdn+4fhlsBCSoBmTAEk/jU35da1QsEJ2BOsjbSurnQ66dRdVGGoI7UHyZYNTXujF5lGZld+8uwE9+iUChP4D3lxvuUb6TLq3cOpK5Xt3FCnkcwMifEa+lF9G7MYdZ/S091NONdAcRdSxaU2mS22e4z3HzOwED7BmZMkmvYrAPh/q2iQm41BnnQTjJbsW5xb2JMOn1a9/Ie+pcCPqQOZcA/tUPqLaFTv+40ThjCWQebT8CaWwR3hj2xPLYUda7WvOctAWDqx5QPTvphh7cAHwk+Z2oOZElsiRWALPPsg6cvD5GqxfYXb5zCSBE+Lb0+4iBbw7HmxHxI/AVWbMlnddxGgjXQd9OlskvqFgSbbOP6uW7TAoWHgTOY7yZ5zzonjGJv5PqSZG+XLJjf2tKCbFG6M8XbZQ7nUAj74Qnx3qbhGMyglrguAn2hHxj3e6qfJo0bC3DcWZlBhh2gHVlhlcHbTQg76Vduj1/slfH5j+FUbj3FVzKqAZi06eAOvy91Wvo4hVQo+4PhFBGVO0Jyw2pjrHzy8KDuGCsCZOtTcQBZQaCvTIAOxHxoMjtt+4vGtgDFqc9wDv1rPmxLHGshRmCsQconLLkF27l1XWr/AIlD11weXypceAdaxuW8wK3Ve7k/9wW7ZhGHOQ7jTvFSELr6jVk039CfCWmVSVJ25RPpOlcYu+0B0uPaaCAHUMhnmfH/ABRS7CccNki1iFKkaSREj1o27jbV7KissucoEjWfDnTU0P8Amd38WiWR1hHZGvdprNUPoW3WWsVcGzXMw7/bP760y9wGyy5bqC4FA0acp5EFefLQzXV7BYe1bI6q2qDTKFAWNIAA25UaT5YnI1JUikYca07ww0ovHdGQR1uHOh1KE6j9U/gffQ9q2V0III3B3p12YnFx5C7NG2KDsUdhxVooMw607wns0nwwpzhPZpkQWTV6vV6jKPzHZFGWaHtLVh4T0WxN5BcVAEOzMwEztA9ozHdSmy0rL19DuIBtXrfNXV/R1j/66vlUHonwhsIpYP23gMSYXTYCRJ33p9hcXfuNkzFZM5lNttOcZl/CpGa4GeFKrLDXqR46xdR7C/lF4i5cKNpZ26q44iLYjVBRq4PIys164dYAZlCknQCAonfamig+vVG9wCJIEmBJ3PcPHSvdYJyyJiYnWO+O6oQ7qmdNyRd03Nsf6mFWj+lbH/Gtf9Rf31Vun14C9bSQGe2cvjlbUf5hSc/pGY/ULrmiJ3gHy9mKLS3LWV7lJ9wA/Gl2MaBbWNdRv5UfhmY3/BbY+JM/IVjHj60hCKu+Y8u4/wABR9lMw/WafQUrTEHNBOiiPU6Cm3DkgFp02HkNz6mamFapUVk2Qs6aXstpB3t8h/GqGvGjh8SysCbbnf7pUAT5H8KtPSTi1vE9WthusiScoJ3IEREzoaj4T0TLsL123DalQ8QJjdSN9PSmzuU3pCxSUYJtnSWLd1A9smSN1JE+vOldnCopZLSF7rntZZJ7pI+Emrrc4blQlSSRqANBpyiuejLBrIuC31ZdmLArlYnMRLaamBReE26Yb6jy2t+xTOCdBMRcJuYnJa+slEBzutsLGVsvZkmToTvVr4ZZy3HT7oIHwj4GnppPxDiNqxd+slTcUENBIlDBBjWYK8queGKWxmn1LaubCL1rNbIPfNL7qgkaxr74o3DXEZGKOGUzBBkR/PypXd3I7qxz+39DcbvdflnOMtDrs07p796a9G7UWifvMT7gF/Ck98y6N3qR47iPnT3h+EHUJbcbrqNQRJzctQQflWjplbsDK9grEYZHGV0Vh3MAw9xpDjeiuFtEYi3ZCXE1GQsq69k9gHLsTyo/FYAW1BR7oOdBrduPo1xVOlxmGxPKpb2Du5SFvzIj6xFYa+FvIfjWyUFJCYSafJDhcGLilixE6Rppp/GhuMdGTfQp1xUGPsydDPJhR3B7DpmV9QDoe/fX3RTGhjCLW6GTySUnT2EfDOC3bShTcV42OUr7xJqsdN+ifEcVdU2L1q1aRZXK7C412NC/YgLOkSdJOswNDr1XoigHkk+TEb9rifDsrYu+i22OVCydetxtWg5SLqHKOWmh2MTcOivGkxVsujISrZWyC4BMTtdVWHlqPE0l/wDyJW4bGHyqSiOzOw+yewqSNwDmcTtIA5iqX0F6SfkxeLhUHKxXLmDRIgtkZgPBR6ilZIpeZW/ZBLzKmblhlptY2qr9FekmHxUC3cQvEwpJB78pYAmCDyB8KecU4raw65nIH3VGrMe5V3JpkJKrB0tuktxhXqpP9c8T/wAq37Lfvr1V40TR+iy/jRkSLWj4PpGgt4Zl9jS04G4IQfIgehrPFFOOGtmthI2ct+0FHyT40ubpCsPqNLu3I/8AdAU7LE6xrEnu8OVQ8MdmvWwh5yWCmDvIKz2T48590PBbdu5csg65SY8CUYR8auVjDKnsiKuEdW43Jk0bVyC8ZH5k915P8wZP+6huPqLjJblhE3cyKzFWXS3IUE+02bu+rphj8N1iqJiHR5ifYcNHrEetdW8OBca5rmYKpnYBMxAHq7H1rQZBVjMcXsJdCdtbqBkPZh+sFtlk8pbfu1ovhKEF88G9IznkV1yZO5IkAd4bcyT3d4arF9SA5RiBGj2yCHB7zlQH9UVPfsZirAwy7HvB3UjmDHvAqEA+BW3FpAwTKqBRAOaV7Os6cjWf/Tzde2cHcTvuKdJB/NMP9JrRrPDlWe0+rM0Z2AGZixgKRzJqi/T5hS2CtOPsYgT5NbuD55aCaTjuFB1IWdFr64y0HDnPagsja9nSSr7+hnz10fYRouXDP2go9BP41mn0aX7lnFIrCLWIRkG0ZgM42/u2HrWkYcTkXm5LeOrfurDlWlbGlO3uCYni/WXVwqubRNwJ1kfaMZCpB79IO9XpeHArFxi2gBA7CzEEgKZE90msdW4Redwcw6wkHuKsY9QRW1YDEC5bS4NmUMPUTWrDCKRzMWbxMkrPmBwFqyuS1bRF7kUKPhvRNDYrHW7UZ3VSdh9o/qqNW9BQ1jiLXQrWbZKNBDuQilTzUasdNtAPGnmgZV6uEcGYIMGD4HuoFeI5zaFv7ZJaRqqIYaR35oX1J5VTIG3bgUFmIAAkkmAB3knaqx9IGthLq6gMACNRDjQz3SF99GGzOJdXRrplLlvO31dpGGUwp0zBkcghSe0BIFddJ7lu7hb4DKwVSTBBhrcPGmxGUULFZ46scl7FP4Lx44Y6jMjkZxOw5svjHLnAqbifTjApdeyt4Zoyhss2yxmO0Sq+pIHjSbiKqUBG50qi3OEm/i1sLo911UE7CSZaPABjHhWeeOL3Zm/xnUz3g912NGvcSxAyi5ctIxY2rBtuZa6wlc06LmAIymRIiTOupKKxnh3Q+xh+NYazaWUKi72zmKm1mYmfFkX3mtoq+mgktV3ftR1M0rdVQNjxIUd9xP8AKwb/ALaj4piGVQLYU3GOVA05ZgkkxyCqx9I513ih27P65J/6Vzf30GcLcu3TcFxrSpNtAFQ5hoXftKSJYBR4JP2q1CQxsWOrS4ASGKQNj9YyrJ8s0+ldW8Wpc2wZZRLAahdoDEaKTMgHUjWl72vqXtMzErcALDstDOrgqVjLAeARtlqbhn1UWGABE5GACi4NySBoH5sOftDmFhCXE4+CypauXCvtZMggkSBNxlBMEGAeY76MB9KV4e5f6y8Ft2inWaMbrBj9Xb3UWiB3b8qa1CFA+kvh7Yi4LapnC2iGNp1TEKLhYMFW5CXEYASrMNtJMRg3ELbWLr25YMjFZZSjFfFG1WRrHjWp9N+OD+lriKrhrfVoHUwSCltjkblHWElDIIWaX9Kst+7abE3bt21bBuZeosLcYIMxU3FImRpERr5VleWOOVSe77GhY3KKozvAY5rbZkYqZkMphkP3lI2rReF9LBibT4m9fZMSi5VtoShuG2oZrgOaBmBJOkAg8oqDjnRLD4uWwNi5buxIysptMdN1+xpzBA8DV26D9H14fh+pLdZdYtcuEDd8o7KgicsKBJ3M6a6XGUM0bRFKWGW34ii/+qV/9P8A/ZT/AGV6tF/p5P8Al8R/0v4V6meCvxhfqX8F+xntno/iCAerIB7/AMRTPgGE6rEiy+p3Mfqg/jVsDDQ7jN/4pVcsEY57n6K/FII+AoMqpA4PUfeNXzYu4V1MA4hM36ikEitQFZl0uw2e3bP3X+YIoHh/EsS9x5xF4gZRHWNA9okwD4j3UOOdNobnx6qZrleqlcGw94+3dumTpNxj6amn+GwgtQSSxPeSfDn509SMrhQ2r4WHfQGUTqAZ28zUy4e2JhFG50A35/OrsGifrF7x76S9LsNaxOGe0SrBmTTQzDg7eQNG4q8Las0aATAil2PuyAMsQ4kz46jTv1FBLIlyFGPwKb/Qlm1bQZQCMQzWlEjKoZ8oHgAw9TTHh6as0k9WoQecSfdPwpYt5mvK7mdZ8ABLQBTnCyFMAyFZm/WILGsUnqZofkW5mvCiGukpcJUsSZmCCZmDzrZ+g+IzYcJ9xivoe0PnHpWPcOUC68CBmJHkTI+FaN9H+NAvNbn20keaH9zH3VugefxZK6j5lzs4O2jM6ooZzLMAAzeZ3NB8FYIty2dOpuMvkh+sT0CuB/hppQFzh4L3GJlLtsIy6jbPLZgZEqwH+EUZ1RNw03bQ6/I3VXWa7dQ6upuEsLiIuwAKhlkkgTAM5jMFw1S91gGUl81u6pglXVXIBOhXOznKQVkzFORppXiaoIWvwzMyvdbOVVkIUZEdSwIzrJzEARE5TmbTYBf0vxATC3hoAbZQDYDMMoAHmRTy/cgVnPS7jy3rqWEIZPbLzKkqYUL3wQTPeBQtmfqcihjbYtNqF18T8IHy+NKOCcLe5xO1luNbhbjF0ClhCZOznBUH6zcg03cz3x40ov8AGmweItXUt9Y75rapMSzgBZPIZopeRNp6Tlf4+VZ0XroJwspjL4uXHvNbROruXdbgRzcgM2knN1g22C1fKyp+L41kXE4f6i4c9q9abKxW5bKtrmBmFF0jwajuH9PcXmNrq7V90I6y4Js20BE5Z7eZvADTnFKw5VjhWTZ9z0c4OUvLwaRXpqi2/pUwwvG09q8CsBnQB7YY/ZmQSdRsDVls9IsKwnr0G57RyHSZnNG2U+41qU4tJp7CXFp0xjdshgQRuIPI+GoqPGYcXEKkkc1Ybqw2ZfEH+YqSxiEYSjKw/RIPyqSjsEXLhr6s5W7ayscwzWmJByqDJFwA6gnYbxykmWFYKA5DNGpAygnwUkwPCT51NXqhCk9JOgFm5cfFI7W7zMGYntKx0HsyCNIG/pVYxfB7f5RrNwKpQhoymYzDL3HKN5rR+OYrLlUeLH00Hxqn3LUEzvNZZ4oOeutx8Jy01exYrFtVUKoCryCgAL4ADSl+Nx9lWJa2xZFALArGttnjVtDlU7+HeKNRtP57VCXLNwyM9uD322kDkCRcHLSnoWAf1ewf/Buft3P91eoj8nuf8VP2bn/9a9RW/iQW8LwxNsMzdW0DMpMjMOYobAYzrHa4R7RgEfdBhZB5wOVQ8Ua8lu6S6wOzoNe0Qv41xwloVY76zZ3TSNHTxu5Dziq5rREHTXbu2/nwpb0Yw5JuwNQ+ukzKrHyp1cEoR4VJ0atwbhEe0J9VT3UMF5hs35BlgLDCZB8OUUasmVKknvrq1mjnv8P31IQDtWgyMjwqGQp3GpO8UWwgEfzr/wCK+YdFHyr64EHffWoD3FPE56t171I+FBY1mi5En6wkeIWY8+VT8ccxuQOZ+16CDNKvyoMAgvMrR7LAbT90gGkZFbNGPHasRYLS6ZGqg6H3fjVw6NYUMWYjQiI8DvSDHqOvtoWEZPa5+2uh/DzNXHhaBAoHdSoQbn7IrM+xixwxt37lo7o5TzytlHvqx9H8R1GIssT9oZvBW7B+BJ9Kl6U4JbPELhII6ztqeRJ5/Fh6UluMZYncj+Fa+Dy2R6Mnumbca+E1UW+kPC2rFt7jszlAWRFJbNAzCTCjWdzVK479MOIuSuFsraXbPc+sf0X2VPnmom0drxoVdmu376opd2VVG7MQoHmToKqfGvpKwlmVtZr78hb0SfG4dD/hDVVMPN9bOIvZsTcIXMsZ4IKtIzNCarBVVg5o2qzWMFbsqTbspamSWb2vhJA8NKW8i7GlYMk4qSaSf7lN6Tccx2NBS5msWjr1ahlDDuZjq/lt4Up/o671lnrD7GYg947P7h7zWiYh7b2YdgTowKxAYgERJ8f8w76ovEFvW77PcMoVC22GxkmQRyOgoI5FP5nM67pcuNOV3H4/C9h2F7NVzi+Da/etrbjOjBknYssuF8JyRPjTcYyLbHuX4naKWcKxfV3rFw/8xbB8JZV+Gejm2k2jm9H/AOkX7ot3SnEW89rE2ipt30DsO/KmaY72sm4P/jFJ+1aFnB2SBfuqXuvEhDdYu9w/eYk5UB0hJOi6ouC8edluYZwD1OZrWUGYtOS1ojv6suqkctKe9CcE8Xb7shuZzbtFzJi1NsHY5QVWARrqx564ZKUpvXx/fuewglS0/wDAm7h7OEt9VbUyLlpmLyxZncgszN7THLr3abaUx6blFa5kAUGxckAQJZmDTH86mjsM2IyRdS04+6pLAjl7QGvpQPG8I7sxIChpC5ixIneTtEsx570epRhXLI8L1WimHj3VOPq2m6AbYXcusqVOugIyGfOhukXSnGYIWrFrF3esgvcYuzSTyAeQFnNAjYCo+HsTcfF3iFS2WRNZhUJDZY3LHSfMVUuIYu5i8SWjt3HAVe6SFRfTQVOmx1Pbhc/PsvoufcHLLbfvx8jYPoZ6cYvF4h7OKu9YOqZ0JS2pDI6AiUUTIede6tYrLegfDEw17CFInqxac8yLi3mmPFrVv3itI4piOrts0xpA8zpW2E1JNmWUadCHG3OtvMeS+7TQfiarGPxxFwKAILATrsZ/cKsmDwrfk127BzXG0/UUwAfjVWx1uXUwdGHwMVPcNFmW5oPh+jXDXxrt4jvoNb2g8hP6Xh+FA3irXCGLLrsMrR4+1NSwRz1w+8P2q+UoyWP+Jc9y/wC6vVLRewj4l0lwl2xdUYi1mJJUZoJIbMBr3x8a9wrESB76xua1DoTiuss259oCDO+mnxEH1pfUR4Y7ppcxL3hb0qPKpOjFybl0xoHBBOYj2QDELAIjvn0ilPD7Ny7cFi3ppmZj9hQYJjmdYA5n1q9cOwC2LSW02Uak7nvJ7yTqfOhxpvcPLJRWkntuTz9wO0b19KwNJ+P41Jp3aHbx99clJ3p5lJLRGXT+TXJOn8786lt7RtXNxPxqFCbHpM6+tKrXE1Kh1unJrGZdSdAddO7upxxbDOwhCB3z8v31S+I22sFEfQurMNIB+sbl4SPfScja3NGJ9jvCWxiL5uPGm6nuEMkeRUz+tVtwmIIcCdIqi8MZjiFIOk9r91Wyxc7QpeKW7By8n3ptws3+ruIs3LRJgbspUgjxiZA86z+4wK6GSJ+daZYxXbWguk/R6zdt3bqWwt/KSrLIzPGmYDRpPeJrTZyer6PxHri9zIeNXx7Op0gAd53P891L8ICkHq9CZOup8akFwTmuHL4sY3864x3FbIRgrZmgxA0mNJO1UkZIwlWlKy59FrgUq+fKmqn6xV3AOiv57/KKsXE2D2oWzcuzpm6xWC6TmPbyj3VnXRFT1TLmDMoW8plWgsjkrlElYgb6yZFNekXS29bsrmdshITsGGPZDSSwPjSZQ81HfxZ3DGoy3cVSruiyW3hOryKIiAGLgZQACAdjCj3Co8eBcQqdQBr5z+FZxb6VKDP1kkakRJ8CdK0HDXRcso4+0oPvE0ueLw90MwZ/GuM41/NirDCCbTAyuo00buP8KCxmCQjqbr9WjvJfTsrlkkToD2Dqdpk0V0pbJbzgwQfDygg6GqjjuN22Rw6HN1YFuApUPnVsxDbaIRpO5HOQ6Lc47cnEl0Sx9VUfSnf3osXDuGomOuqoNy52HFxWyJaJElrmRu2xMHLsddgdLzgMFbSyLJKwqkSSFYsObMI1PeKo30eMDZut9p7ksSdT2QdfUn30Z0v441i1nVhnJCgHn3z36TSnF6qu3sj0GOlDVwPcNxqwGFgXLy3ChcKSx7CgsSC4M6KTvypTxzilzE3HwyOWAGVM3ZMnKHZssSBqY0mPGknDfpGZmDX7CMqkqlwDW0HBXUxqcubaJAOlQ9KmvYXNcJQNfTKhVpJRhL3Fy6RrAM/a8KuWOXpXL/sjzJ79v5EnSbFrPUW2m2nZkHQ5TA8/xOtEdBOHE3evYdi2ruPFlXl5Zh6kUjwFpWcdY2VB7RGrR3KOZPwq0XOmFq2oSxaMdU9vUwBnKQRzMBTvuWo8ilGHhwVt8v8APiZ4tSlqkzSOE3DbdSdMl60pY8lVAxJ/a595q58Wz3lCggAGSOZ8jyO9Yvw/i1/F2710tbRSxz2wHJ0tNlkAEkE8p7UECDWpdHbh/J7SsxYqgUkkEmBzI0JiKz9InjvFJ78/7DzVLzI6x3T3AWFNi7cuJcVcuV7V0E6QDIUgjxBIqm4rpjZY9lkP+Jh80q2dI+BYfG2uqvpO+Vxo6HvVuXlsedYl0r6L4nAXsks9s/m7ijRh3EcmHMVuabEJ0aDa6UIREWxG31o0961LgMZhnLNc6tXP6cyIHMGN6yP8uvjdT6qa6HGbg9pR8RVODaotSSNm6vCd6ftn99erG/6eP3B7/wCFeoPBkFriWzoF9Fz4xFxF+4bVlvYCgdY4+9rog7jBnuiCdJsdB7Nq0LViVC6hzBcttq0CfLbwqHoZxS41sW2ykoAgIBgrGmmkHT4VasPiOTbeGu3hTpx3pgQk1ujjhnDuoUKsmR2yYliOZNGWjEhgfAb6eEVD+UKSYMDbzrnEY4AQYPgKqqLtt7k9vETo0zyHh4eNSWbh+0DNKr3GIIi03vn4Hb31Jh+M2WYBmZGPJlIE+cR8ahQ6Xb1r6xr5Yjvmo8QxXyOxq6K7guOViNGCx4SdKQ8cwEo+a31zKGdASVMxMAgErMeO+1Pbroe0RtpNDcRvFYYKTE7CdI8KBqwltwUezftoqMAEntEMZIOnZJIG221NrfFLWf2x8aacN4La6s5xnLySdokzCwZWKU8U4BctPnt9u35ksvmCYI8aXDFGMmySk2GWcUBlYkBZ3nTeoek/StLFkm0yvdYgIu48WaOQ+NLeLcWSxYLvuTlAPskkHc7AaGqpbshxn011mQQZ7o0ptHO6zrPBWmK3f8CPF8P6xjdeDcuOWJgDVjJIA0Gpphgej6kEuNBy/CunUZp7tB58/wCfCjBxCFyg69/dUONPPOXcg4Nw/q7zhRCdXkgDXM3anTuB2/SNMOkeCRkVWVSRlYmARmysGA8uz76CtYjKSw1JMzOugA+Y3qTjmMLBCTMpPvZv3fCq027NUepyLE4PnYrzYSyDBRfMAU2wvSHqLWUqWXXKRJy+DeGuh9KR49GnMKg4XZxVx8tgFz3RoPU7VUkpKmaeknkjJSi79mF8U4lcu2wGmScxH3RyHhE++arV/DuQXytlBgtBgEzAJ2EwY8jV1boVj2kNaRSftF9D5ZQY9YpIGxXD8QettAEjK9p1DWrtufZI2ZfHkddxRQ24NkFPU5S7i/hXSLEYdGt2rmVHMsMqNJiJlgSDHMVFxbilzEENcI0EAAQBtJ8zVm6a8Hw7WMPj8OnUDESDh25FTBe3GmSfKZWBrFVP8hcvkAzN+iQflt60Sa5NPmqgrA8SFtUEZoc3CpAAzxlTXcwJPLVqdfSBYZGw9tjmZbUFoAzQxjQDkIHPQDuqvHDZCwudllJBB3kfxH8zVr+kHiqX7eHvCz+dzMruWD5FIXJlVssSN9T4iqrzKSJe1Mp4BJknU+MU34f0ZxF383YuHvJGUA7ES8DlWidAei3VouIvIq3TqiqMotqQIBA3bnrJ176t1tIpcsj4Rohh7sqPQjonfsLe63qwbnVlcpLEG2Se1pHMbE86unCeH9SmRdpJHrqfOTJnxr06yKaWtRSFBa9fcbKK06QR31pL0pv4Z7LWr/aBGgX2wRsy9xHfX3pvwnEXbUYW91VydTyZYggkAleRkVluM6J8WskuA9zmWS4Lk+hOY+6tK3RllHSz39HtmMJdCzpmAJjlOXSaD/qq9242VwCe121PM7VH/WLF2myXQVPc65G9zD8KMwnSO5mzaAxGqhh8CtDUo8F7SIv6gX/+JZ97fur1Nv6zP3p+w/8Aur1TVMmmJpHR3BIlpQs5icx1199PUzBeX8Khs4IADKAIGg5R4d1SBNNyNO/enSduxaVIlAPcP4V0qD7orjKfvHuNdKWnfnz8PKqLJ1Uc1r6cLbbdfhUVu+3cDE+HrUlrE7AiKhAm1YAEL2fL8RzqPG40WULXoySAWGo1Majca900Zh43pH0mxlq5bNsMjHP2gDMZZmY2MxQyelWXGOqVBGGW1eGa3cW4ncpB18YMjyNE27UH5TWd2OFMt09XtuIMH+fGrLw2/iU3uFvBhm+J1+NBGafYOUGnVliaDsKjIqLC4wnR1yk8xJWfI7e+ucYC2hJy+A1kajf+I3mRR2LaK9026KflVh7dpltuxUmRoYYE7bGJo9OimFFpbfVBVUQMuke78ZplgUkHuXsxliNARsSCII2oxbY51AHjjK9SuzNeMfR1dMmxeB10VgR6ZhM+6gOGfR5ic4N0qoGv3vgN61llr4sc6lCP0eJO0iidJ+ij3bCLbEva9nSCwO4nYd/pSK90IxV1bPZCRbytnO0O0aCZ0M1qeKxaW9DOo5D8fSgLXE3Zo6oRz7Q0HKMpJJ1G6r+FSi59NCUrfcqPDPo1UEG87XPADKD4Hn8qt+D4MlpcqW1QDkqgD4CmWFuZhqpXz5/uqZl0qUNx4oQ9KoWBMvIEdxqtdPujtrGWCDIKGVYe0hOnuP8AOsVbrgpfe7JOkgiCDsQdx7qjGVZiXSHi/wBVYS8q58OpCA6NrEWyB9kaH/BHOqgmOuKzOrsrNOYqYJkyRptTTpopGNvJqQrZVnUkCI987eQphwTomwAu3gF2KoRJJJAXMvMkkAJIJMAxrQynDErk+S4xlN0ux7o10Wt32+ue5mCi46KgMBjKhrmfslgJiAe1V5s4WxiGsWyjMuHg2icpy5AoZLgzZgZKkhhyB81fAsOcIX1drjH6wEdq4RJL2X9m5BJBUwSBIj7Tvg+LR7jugjRVkqVJiSZB1BBaIPdWH9Rkllpen8/EbI4YqPuWa3cEVGz0PaY13dugCaeEkTLc1plgr3KqthuLob3V5hmiY+dWXBsDUiySVBWISQaGw1vMGH2l+WsGPhvyooUvxJKPmUwfmO6mJ0xUo6kS4zhy3AFe2LinQh1lRpOoYajSPUad1d4h9HGDuSRYNpvvWWKf5dV+FXbCsrqGGvnqQeY8K7a1TUZmZh/6YWf+YxH7Nv8A2V9rTPycV6rKBkQ5o7zO3d/PxqZmBEGCZrldv57q7f2V/nkasogbD6EgaTvzG3dXBtmJBOh05+fzoixzqEb/AOI/KpRZ5Sy8gQR67/xrq9jUthi0abyYVfM/gJNTruPP99VLpF+ZX9cf6xVSdIKCthWPxbYkQvYt84kMw7onsj4nw2oRcOEGVRArrAeyK7xW1ZueTfGKithfwHiDPxUIPzaWTab+8uFXE+WRR/iNaItgATArLeh39tvf36/NK1nEezTcfBkz7SBbglTEAn3UvfEKq5HaCdPHTmPKmJ2pZc/Pej0UhSGOHtCNAAN9NPXSgcT0gsoxth+suDdLcMR+sZhfUivvSX+yv+qf9JrPehH5j30rJkcaodixqVtl2bpC59lFHmSfwFdWuON9pFI/RkH47/ClFjYV2fb9KDxJD/Ch8Cz4TEJcEo0xuOYPcRyr2KxqoO24Xw5nyUamqnh/z5/uj/rFA8H3vf3hpnisT4Kst93pBYtjMWb0tXWP+VDTDCYtLqBkYMrbEGQapxpn0I2xP9//APXbqQyNsrJjUVsOrgpdxEwhY7bfCmt7elHSH8w395b/ANdMlwJXJmWFwbWbuIxeJwjhbtw5Ha3n7BICiACbWw1Mb0vudIiVuXEUZ0zsAfZtBJVWbvMCFHNrj8hWzf8At3P1G+Rr858R9nE/3n/clZcvTw8TffgdGbiti82LycRQOl027aKoNsBMwdRLMxZSdJUKRHsk110OJa0HknMZJOpM8ye+qX9G/wDa/wD43+VXX6Pf7OnlVRxKEpRXCr7jYTclbLOzwKTXsW9+4bVs5VHtv3foj9L5U0xmxpT0d9hv71/nRyGRCv6r2wM6CLg1DmS0+Z+VO+DYuRB0I0I8antbUvsf2h/T5VRbLIrzQ2MFSWK+YramMV3K5x7il7CJ+U2YZU1uoZIa39oiDowGoPnvV4R5AIOh1FUzj39lvf3b/wCk1aOEfmrf6g+QpmPgRmW4ZNer1epgk//Z',width=400,height=400)